# FPGA Deploymnet

In this lab you will deploy the custom image classification model developed in Lab 1 to an FPGA powered cloud services.

Project Brainwave is a hardware architecture from Microsoft. It's based on Intel's FPGA devices, which data scientists and developers use to accelerate real-time AI calculations. This FPGA-enabled architecture offers performance, flexibility, and scale, and is available on Azure.

FPGAs make it possible to achieve low latency for real-time inferencing requests. Asynchronous requests (batching) aren't needed. Batching can cause latency, because more data needs to be processed. Project Brainwave implementations of neural processing units don't require batching; therefore the latency can be many times lower, compared to CPU and GPU processors.

You can reconfigure FPGAs for different types of machine learning models. This flexibility makes it easier to accelerate the applications based on the most optimal numerical precision and memory model being used. Because FPGAs are reconfigurable, you can stay current with the requirements of rapidly changing AI algorithms.

Today, Project Brainwave supports:

- Image classification and recognition scenarios
- TensorFlow deployment
- DNNs: ResNet 50, ResNet 152, VGG-16, SSD-VGG, and DenseNet-121
- Intel FPGA hardware

Using this FPGA-enabled hardware architecture, trained neural networks run quickly and with lower latency. Project Brainwave can parallelize pre-trained deep neural networks (DNN) across FPGAs to scale out your service. The DNNs can be pre-trained, as a deep featurizer for transfer learning, or fine-tuned with updated weights.

To deploy FPGA service you need to:
- Create a service definition
- Register the model 
- Deploy the service with the registered model


## Create a service definition

A service definition is a file describing a pipeline of graphs (input, featurizer, and classifier) based on TensorFlow. The deployment command automatically compresses the definition and graphs into a ZIP file, and uploads the ZIP to Azure Blob storage. The DNN is already deployed on Project Brainwave to run on the FPGA.

Here we use the TF.Keras model trained in the lab 1 in the classifier stage.



### Connect to AML Workspace

In [9]:
# Check core SDK version number
import azureml.core
import os
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.6


In [2]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /data/home/demouser/notebooks/aml_config/config.json
jkamlworkshop
jkamlworkshop
southcentralus
952a710c-8d9c-40c1-9fec-f752138cc0b3


### Retrieve the top model from the model registry

In [5]:
from azureml.core.model import Model
from tensorflow.keras.models import load_model
import tensorflow as tf


# Retrieve the model file
model_name = 'aerial_classifier'
model_path = Model.get_model_path(model_name, _workspace=ws)

# Rehydrate the model
model = load_model(model_path)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 526,086
Trainable params: 526,086
Non-trainable params: 0
_________________________________________________________________


### Define a service

Define input tensors node.

In [6]:
import azureml.contrib.brainwave.models.utils as utils
in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)


(?, 224, 224, 3)


Define **resnet50** featurizer. It has to be the same model as used in Lab1 to create bottleneck features.


In [11]:
from azureml.contrib.brainwave.models import QuantizedResnet50
model_path = os.path.expanduser('~/models')
bwmodel = QuantizedResnet50(model_path, is_frozen = True)
print(bwmodel.version)

1.1.2


Define the FPGA pipeline

In [17]:
from azureml.contrib.brainwave.pipeline import ModelDefinition, TensorflowStage, BrainWaveStage, KerasStage

model_def = ModelDefinition()
model_def.pipeline.append(TensorflowStage(tf.Session(), in_images, image_tensors))
model_def.pipeline.append(BrainWaveStage(tf.Session(), bwmodel))
model_def.pipeline.append(KerasStage(model))

model_def_path = './model_def'
model_def.save(model_def_path)
print(model_def_path)

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.
INFO:tensorflow:Restoring parameters from /home/demouser/models/msfprn50/1.1.2/resnet50_bw


FailedPreconditionError: Error while reading resource variable dense_1_1/bias from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/dense_1_1/bias)
	 [[Node: dense_1_1/bias/Read/ReadVariableOp = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1_1/bias)]]
	 [[Node: dense_2/bias/Read/ReadVariableOp/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_18_dense_2/bias/Read/ReadVariableOp", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_1_1/bias/Read/ReadVariableOp', defined at:
  File "/anaconda/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/py36/lib/python3.6/runpy.py"